In [1]:
# generate explicit model files for Access Control (10 servers, p_free=0.06)
# states: (f, q) with f in [0..10], q in [1,2,4,8]
# actions: 0=reject, 1=accept (accept invalid at f=0; we omit it)
# rewards: only on accept when f>0: reward = q

import math
from pathlib import Path

# ---- parameters (match your env) ----
N_SERVERS   = 10
PRIORITIES  = [1, 2, 4, 8]
P_FREE      = 0.06
OUT_PREFIX  = "access_control"  # files: access_control.tra, .tra.rew, .lab
INIT_ALL_Q  = True              # label all four (f=10, q=*) as initial

# ---- enumerate states and ids ----
states = []
id_of = {}
for f in range(N_SERVERS + 1):
    for q in PRIORITIES:
        sid = len(states)
        states.append((f, q))
        id_of[(f, q)] = sid

# ---- build transitions (src, dst, action, prob) and per-transition rewards ----
transitions = []
rewards = []

def binom_prob(n, k, p):
    return math.comb(n, k) * (p ** k) * ((1 - p) ** (n - k))

for f, q in states:
    sid = id_of[(f, q)]
    # action 0: reject (always available)
    f_pre = f
    busy_pre = N_SERVERS - f_pre
    for k in range(busy_pre + 1):
        f_next = f_pre + k
        if f_next > N_SERVERS: 
            continue
        p_k = binom_prob(busy_pre, k, P_FREE)
        for q_next in PRIORITIES:
            did = id_of[(f_next, q_next)]
            prob = p_k * (1.0 / len(PRIORITIES))
            if prob > 0.0:
                transitions.append((sid, did, 0, prob))
                rewards.append(0.0)  # reject pays 0

    # action 1: accept (only when f>0)
    if f > 0:
        f_pre = f - 1
        busy_pre = N_SERVERS - f_pre
        for k in range(busy_pre + 1):
            f_next = f_pre + k
            if f_next > N_SERVERS: 
                continue
            p_k = binom_prob(busy_pre, k, P_FREE)
            for q_next in PRIORITIES:
                did = id_of[(f_next, q_next)]
                prob = p_k * (1.0 / len(PRIORITIES))
                if prob > 0.0:
                    transitions.append((sid, did, 1, prob))
                    rewards.append(float(q))  # accept pays the current priority

# ---- write .tra (explicit MDP) ----
tra_path = Path(f"{OUT_PREFIX}.tra")
with tra_path.open("w") as f:
    f.write("mdp\n")
    f.write(f"{len(states)} {len(transitions)}\n")
    for s, d, a, p in transitions:
        # src dest action prob
        f.write(f"{s} {d} {a} {p:.12f}\n")

# ---- write .rew (per-transition, same order as .tra lines) ----
rew_path = Path(f"{OUT_PREFIX}.tra.rew")
with rew_path.open("w") as f:
    f.write(f"{len(transitions)}\n")
    for r in rewards:
        f.write(f"{r:.12f}\n")

# ---- write .lab (init states + human labels) ----
lab_path = Path(f"{OUT_PREFIX}.lab")
with lab_path.open("w") as f:
    f.write("#DECLARATION\n")
    f.write('0 "init"\n')
    f.write("#END\n")
    # label initial states: (f=10, q=*). storm allows multiple 'init' states.
    if INIT_ALL_Q:
        for q in PRIORITIES:
            sid = id_of[(N_SERVERS, q)]
            f.write(f"{sid} init\n")
    else:
        sid = id_of[(N_SERVERS, PRIORITIES[0])]
        f.write(f"{sid} init\n")

print(f"wrote: {tra_path}, {rew_path}, {lab_path}")
print(f"states: {len(states)}, transitions: {len(transitions)}")

wrote: access_control.tra, access_control.tra.rew, access_control.lab
states: 44, transitions: 2096
